# Cross-Validation and the Bootstrap

## The Validation Set Approach

In [21]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, LeaveOneOut, KFold
from sklearn.preprocessing import PolynomialFeatures

In [3]:
auto = pd.read_csv('./Downloads/Auto.csv', index_col=0)
auto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 1 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    int64  
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   year          392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
dtypes: float64(3), int64(5), object(1)
memory usage: 30.6+ KB


In [4]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
1,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
2,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
3,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
4,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
5,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [5]:
train = auto.sample(196, random_state=1)
test = auto.drop(train.index)

In [6]:
est = smf.ols('mpg~horsepower', train).fit()
pred = est.predict(test)

mean_squared_error(test['mpg'], pred)

23.361902892587235

In [7]:
est = smf.ols('mpg~horsepower+np.power(horsepower, 2)', train).fit()
pred = est.predict(test)

mean_squared_error(test['mpg'], pred)

20.252690858350192

In [8]:
est = smf.ols('mpg~horsepower+np.power(horsepower, 2)+np.power(horsepower, 3)', train).fit()
pred = est.predict(test)

mean_squared_error(test['mpg'], pred)

20.325609365878563

## Leave-One-Out Cross-Validation

In [13]:
lin_reg = LinearRegression()
loo = LeaveOneOut()
-cross_val_score(lin_reg, auto[['horsepower']], auto['mpg'],
                 cv=loo, scoring='neg_mean_squared_error').mean()

24.231513517929226

In [16]:
res = []
for i in range(1, 6):
    poly = PolynomialFeatures(i)
    cv_score = -cross_val_score(lin_reg, poly.fit_transform(
        auto[['horsepower']]), auto['mpg'], cv=loo, scoring='neg_mean_squared_error').mean()
    res.append((i, cv_score))

In [20]:
res

[(1, 24.231513517929226),
 (2, 19.24821312448939),
 (3, 19.334984064114092),
 (4, 19.424430309411886),
 (5, 19.033211842978396)]

## k-Fold Cross-Validation

In [22]:
lin_reg = LinearRegression()
kf10 = KFold(n_splits=10, random_state=1, shuffle=True)

res = []
for i in range(1, 11):
    poly = PolynomialFeatures(i)
    cv_score = -cross_val_score(lin_reg, poly.fit_transform(
        auto[['horsepower']]), auto['mpg'], cv=kf10, scoring='neg_mean_squared_error').mean()
    res.append((i, cv_score))

In [23]:
res

[(1, 24.09767573188306),
 (2, 19.17888986488974),
 (3, 19.213859523719318),
 (4, 19.212807016847393),
 (5, 18.75798588326042),
 (6, 18.641065858861953),
 (7, 18.82081002496441),
 (8, 18.975736690279508),
 (9, 18.937579150541712),
 (10, 18.793495362139453)]

# The Bootstrap

## Estimating the Accuracy of a Statistic of Interest

In [24]:
portfolio = pd.read_csv('./Downloads/Portfolio.csv', index_col=0)
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 1 to 100
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       100 non-null    float64
 1   Y       100 non-null    float64
dtypes: float64(2)
memory usage: 2.3 KB


In [25]:
portfolio.head()

,X,Y
1,-0.895251,-0.234924
2,-1.562454,-0.885176
3,-0.417090,0.271888
4,1.044356,-0.734198
5,-0.315568,0.841983


In [26]:
def alpha(data):
    X = data['X']
    Y = data['Y']

    return (np.var(Y)-np.cov(X, Y)[0, 1])/(np.var(X)+np.var(Y)-2*np.cov(X, Y)[0, 1])

In [27]:
alpha(portfolio)

0.5766511516104116

In [28]:
np.random.seed(1)

alpha(portfolio.sample(frac=1, replace=True))

0.4504820492455901

In [29]:
np.random.seed(1)

def boot(data, R):
    res = np.empty(R)
    for i in range(R):
        res[i] = alpha(data.sample(frac=1, replace=True))
    return res

In [30]:
results = boot(portfolio, 1000)

In [31]:
print('Mean:', results.mean())
print('Standart deviation:', results.std())
print('Variance:', results.var())

Mean: 0.5794285126180528
Standart deviation: 0.09405177965236293
Variance: 0.00884573725577663


## Estimating the Accuracy of a Linear Regression Model

In [32]:
def boot_lm(data):
    X = data[['horsepower']]
    y = data['mpg']
    
    lin_reg = LinearRegression()
    lin_reg.fit(X, y)
    
    return lin_reg.intercept_, lin_reg.coef_[0]

In [33]:
pd.Series(boot_lm(auto), index=['Intercept', 'Coefficient'])

Intercept      39.935861
Coefficient    -0.157845
dtype: float64

In [34]:
np.random.seed(1)

pd.Series(boot_lm(auto.sample(frac=1, replace=True)), index=['Intercept', 'Coefficient'])

Intercept      39.658479
Coefficient    -0.155898
dtype: float64

In [35]:
np.random.seed(1)

res = []
def boot_lm_mult(data, R):    
    for i in range(R):
        res.append(boot_lm(data.sample(frac=1, replace=True)))
    return res

In [36]:
results = boot_lm_mult(auto, 1000)

In [37]:
print('Mean:', np.mean(results, axis=0))
print('Standart deviation:', np.std(results, axis=0))
print('Variance:', np.var(results, axis=0))

Mean: [39.96338479 -0.1583057 ]
Standart deviation: [0.82526625 0.00713573]
Variance: [6.81064388e-01 5.09187058e-05]


In [38]:
est = smf.ols('mpg ~ horsepower', auto).fit()
est.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,39.9359,0.717,55.660,0.000,38.525,41.347
horsepower,-0.1578,0.006,-24.489,0.000,-0.171,-0.145


In [39]:
def boot_lm2(data):   
    est = smf.ols('mpg ~ horsepower + np.power(horsepower, 2)', data).fit()
    
    return est.params.to_list()

In [40]:
np.random.seed(5)

res = []
def boot_lm2_mult(data, R):    
    for i in range(R):
        res.append(boot_lm2(data.sample(frac=1, replace=True)))
    return res

In [41]:
results = boot_lm2_mult(auto, 1000)

In [42]:
print('Mean:', np.mean(results, axis=0))
print('Standart deviation:', np.std(results, axis=0))
print('Variance:', np.var(results, axis=0))

Mean: [ 5.70313029e+01 -4.68655549e-01  1.24070402e-03]
Standart deviation: [2.07113749e+00 3.33196933e-02 1.20920217e-04]
Variance: [4.28961052e+00 1.11020196e-03 1.46216988e-08]


In [43]:
est = smf.ols('mpg ~ horsepower + np.power(horsepower, 2)', auto).fit()
est.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,56.9001,1.800,31.604,0.000,53.360,60.440
horsepower,-0.4662,0.031,-14.978,0.000,-0.527,-0.405
"np.power(horsepower, 2)",0.0012,0.000,10.080,0.000,0.001,0.001
